In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.2
3.2.1


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

In [5]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(1)

In [7]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [9]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [11]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x00000207FB4C5F80>)
ScalarOutcome('Total Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_over at 0x00000207FB4C5F80>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over at 0x00000207FB4C5F80>)


In [13]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:46<00:00,  4.33it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [15]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['Expected Annual Damage', 'Total Investment Costs', 'Expected Number of Deaths'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,53,298.098615,1.0,0.516423,326.526202,10.0,0.370765,298.435954,1.0,0.432825,...,7,10,5,9,1,5,4,4,0,dikesnet
1,115,278.626880,1.5,0.161402,320.833873,1.0,0.281242,93.705103,1.0,0.079883,...,7,10,5,9,1,5,4,5,0,dikesnet
2,85,98.005136,1.5,0.897363,32.006922,10.0,0.111860,49.295972,1.5,0.779875,...,7,10,5,9,1,5,4,6,0,dikesnet
3,110,69.700200,10.0,0.863577,266.721792,10.0,0.677677,112.572178,10.0,0.536286,...,7,10,5,9,1,5,4,7,0,dikesnet
4,15,131.393226,10.0,0.562096,197.375032,10.0,0.196384,320.311427,1.5,0.394689,...,7,10,5,9,1,5,4,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,128,272.649957,1.5,0.498659,74.649479,1.0,0.171085,177.031112,1.0,0.243273,...,9,2,9,6,10,0,2,49,3,dikesnet
196,57,75.397619,1.0,0.848505,341.021847,1.5,0.641632,206.060300,1.0,0.726545,...,9,2,9,6,10,0,2,50,3,dikesnet
197,55,59.098215,1.5,0.932718,279.754844,1.0,0.256118,325.683068,1.0,0.372335,...,9,2,9,6,10,0,2,51,3,dikesnet
198,97,67.984318,10.0,0.370027,114.779480,1.5,0.902428,285.081281,10.0,0.288803,...,9,2,9,6,10,0,2,52,3,dikesnet


In [17]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Expected Annual Damage,Total Investment Costs,Expected Number of Deaths
0,0.000000e+00,1.978200e+09,0.000000
1,2.124514e+06,1.978201e+09,0.000546
2,0.000000e+00,1.978200e+09,0.000000
3,0.000000e+00,1.978200e+09,0.000000
4,0.000000e+00,1.978200e+09,0.000000
...,...,...,...
195,1.665752e+07,1.417232e+09,0.003346
196,0.000000e+00,1.417231e+09,0.000000
197,1.079502e+07,1.417232e+09,0.001689
198,0.000000e+00,1.417231e+09,0.000000


In [19]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [25]:
lever_settings = {lever.name: 0 for lever in levers}
Base_Case = Policy("Zero Policy", **lever_settings)

In [27]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 1000
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies = [Base_Case])

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
100%|██████████████████████████████████████| 1000/1000 [03:06<00:00,  5.36it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [29]:
experiments, outcomes = results

In [31]:
from ema_workbench import save_results
save_results((experiments, outcomes), './data/results.tar.gz')

[MainProcess/INFO] results saved successfully to C:\Users\maxva\Downloads\epa141A_open-master\epa141A_open-master\final assignment\data\results.tar.gz


In [33]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Expected Annual Damage,Total Investment Costs,Expected Number of Deaths
0,2.889173e+09,0.0,2.043583
1,1.446423e+09,0.0,1.461669
2,2.913219e+08,0.0,0.524921
3,3.582799e+09,0.0,2.018168
4,6.463175e+08,0.0,1.002809
...,...,...,...
995,9.963240e+07,0.0,0.120246
996,3.053333e+09,0.0,2.910806
997,2.046713e+09,0.0,3.762321
998,1.946510e+08,0.0,0.237261
